## TODO: [09/09/2020]
* verify that consensus sequences are identical across samples 
    * use mafftt to do MSA on the consensus sequences from github -> aligned fasta file
    * Install Genious, drag and drop the aligned file to verify that sequences are identical
* enforce a frequency cutoff: filter variants with frequency less than 0.5 and greater than 0.03
* Apply Levene's test on all positions between each of the outbreaks 
* create DF of deletions and their corresponding counts per location, send csv to KG and MZ
    
* repeat clustering but of sample-variant pairs (think about it)
* add cols REF and ALT one-hot encoded (no need to aggregate anything, keep sample variants as is)
* TABLE: for a given pos, how many times does it occur per outbreak
* VISUAL: for a given set of pos that are identified across all outbreaks, look at boxplot for each outbreak at each position (do they align, can you discern between them)
    * make boxplots horizontally aligned for a given x value
* TABLE: test the difference in the variance of the mean frequency for each position between outbreaks (Levine test)
    * tests for heteroscedasticity between two populations 
    * do a t-test to see if mean frequencies are different
* Repeat analysis on variant-meta data looking at insertions and deletions separately

### DATA LOADING

In [1]:
import collections  # hash maps and counters
import re  # regular expressions
import numpy as np  # linear algebra
import pandas as pd  # data processing
from path import Path  # file handling
import plotly.graph_objects as go  #viz

In [2]:
from scripts.kmeans import *

In [3]:
sample_ids = [1110, 1927, 1940, 1941, 1954, 1959, 1960, 1964, 
              1970, 1957, 1929, 1951, 1076, 1066, 1945, 1955,
              1093, 1939, 1961, 1085, 1952, 1090, 1963, 1099,
              1104, 1044, 1924, 1926, 1943, 1944, 1948, 1950,
              1953, 1962, 1925, 1928, 1935, 1936, 1111,
              1107, 1114, 1933, 1938, 1946, 1971, 1930, 1947,
              1967, 1117, 1112, 1113, 1115, 1108, 1118]

In [4]:
# labels
calm_ids = [1076, 1066, 1093, 1085, 1090, 1099, 1104, 1044]
snf_ids = [1940, 1941, 1954, 1959, 1960, 
           1964, 1970, 1957, 1951, 1955, 
           1961, 1952, 1963, 1948, 1950, 1953, 1962, 1971]
snf_new_ids = [1927, 1929, 1945, 1939, 1924, 1926, 1943, 1944, 1925, 
               1928, 1935, 1936, 1933, 1938, 1946, 1930, 1947, 1967]
sd_cnty = [1110, 1111, 1107, 1114, 1117, 1112, 1113, 1115, 1108, 1118]

In [27]:
# x = 'SEARCH-1076*.tsv'
# !find /home/gk/analysis/ -type f -path '*variants*' -name $x

In [29]:
seq_sum = pd.read_csv("/Users/alaa/Documents/scripps/data/COVID_sequencing_summary - samples-sequenced.csv")
seq_sum.head()

,Sequence date,SEARCH SampleID,Original sampleID,Sample collection date,Location,Zipcode,Country,Genome,Library,Run,...,Percentage aligned,Deletions (gaps),Missing # nt,% complete (CDS),Amplicon scheme,Ct value,What to do?,PrimalSeq 3' barcode,PrimalSeq 5' barcode,Notes
0,2020-03-12,SEARCH-0001-SAN,PC00101P,2020-03-11,San Diego,NaN,USA,Complete,L1,Run1,...,NaN,0,0.0,100.0,V1,NaN,submit to GISAID,NaN,NaN,NaN
1,2020-03-12,SEARCH-0001-SAN,PC00101P,2020-03-11,San Diego,NaN,USA,Complete,L1,Run1,...,NaN,0,0.0,100.0,V1,NaN,submit to GISAID,NaN,NaN,NaN
2,2020-03-23,SEARCH-0002-SAN,MG-0987,2020-03-18,San Diego,NaN,USA,Complete,L1,Run1,...,NaN,0,0.0,100.0,V1,NaN,submit to GISAID,NaN,NaN,NaN
3,2020-03-23,SEARCH-0002-SAN,MG-0987,2020-03-18,San Diego,NaN,USA,Complete,L1,Run1,...,NaN,0,0.0,100.0,V1,NaN,submit to GISAID,NaN,NaN,NaN
4,2020-03-31,SEARCH-0003-SAN,DP-pos-9,2020-03-14,San Diego,NaN,USA,Not Complete,L1,Run1,...,NaN,0,893.0,97.0,V3,NaN,submit to GISAID,NaN,NaN,NaN


In [30]:
seq_sum['sample'] = seq_sum['SEARCH SampleID'].apply(lambda x: x.split('-')[1]).astype(int)

In [31]:
seq_sum = seq_sum[['sample', 'Sequence date', 'Sample collection date']]

In [32]:
def get_sample_paths(sample_ids: list) -> list:
    sample_pths = {}
    for idx in sample_ids:
        search_str = f'SEARCH-{idx}*.tsv'
        filepth = !find /home/gk/analysis/ -type f -path '*variants*' -name $search_str
        sample_pths[idx] = Path(filepth[0])
    return sample_pths
# sample_pths = get_sample_paths(sample_ids)

In [33]:
# len(sample_pths), len(sample_ids)

In [34]:
# Loading Variants Data
# variants_df = pd.concat((pd.read_csv(pth, sep='\t').assign(sample=idx, path=pth) for idx, pth in sample_pths.items()))
variants_df = pd.read_csv("/Users/alaa/Documents/scripps/data/variants_outbreak.csv")
print(variants_df.shape)
variants_df.head()

(2466, 21)


,REGION,POS,REF,ALT,REF_DP,REF_RV,REF_QUAL,ALT_DP,ALT_RV,ALT_QUAL,...,TOTAL_DP,PVAL,PASS,GFF_FEATURE,REF_CODON,REF_AA,ALT_CODON,ALT_AA,sample,path
0,NC_045512.2,241,C,T,0,0,0,1651,124,43,...,1651,0.0,True,NaN,NaN,NaN,NaN,NaN,1110,/home/gk/analysis/2020.06.29.hCoV19/variants/i...
1,NC_045512.2,854,C,T,3333,1468,49,2172,963,48,...,5505,0.0,True,cds-YP_009724389.1,CCT,P,TCT,S,1110,/home/gk/analysis/2020.06.29.hCoV19/variants/i...
2,NC_045512.2,854,C,T,3333,1468,49,2172,963,48,...,5505,0.0,True,cds-YP_009725295.1,CCT,P,TCT,S,1110,/home/gk/analysis/2020.06.29.hCoV19/variants/i...
3,NC_045512.2,2143,C,T,0,0,0,4713,2401,43,...,4714,0.0,True,cds-YP_009724389.1,CCC,P,CCT,P,1110,/home/gk/analysis/2020.06.29.hCoV19/variants/i...
4,NC_045512.2,2143,C,T,0,0,0,4713,2401,43,...,4714,0.0,True,cds-YP_009725295.1,CCC,P,CCT,P,1110,/home/gk/analysis/2020.06.29.hCoV19/variants/i...


In [35]:
freq_filter = (variants_df['ALT_FREQ']<=0.5) & (variants_df['ALT_FREQ']>=0.03)
variants_df = variants_df.loc[freq_filter]

In [36]:
# Outbreak Labeling

variants_df['outbreak'] = 'unknown'
variants_df.loc[variants_df['sample'].isin(calm_ids), 'outbreak'] = 'CALM'
variants_df.loc[variants_df['sample'].isin(snf_ids), 'outbreak'] = 'SNF'
variants_df.loc[variants_df['sample'].isin(snf_new_ids), 'outbreak'] = 'SNF NEW'
variants_df.loc[variants_df['sample'].isin(sd_cnty), 'outbreak'] = 'SD County'

variants_df.loc[variants_df['outbreak']=='unknown']

,REGION,POS,REF,ALT,REF_DP,REF_RV,REF_QUAL,ALT_DP,ALT_RV,ALT_QUAL,...,PVAL,PASS,GFF_FEATURE,REF_CODON,REF_AA,ALT_CODON,ALT_AA,sample,path,outbreak


In [37]:
merged = pd.merge(variants_df, seq_sum, on='sample')
print(merged.shape)
merged.head()

(1684, 24)


,REGION,POS,REF,ALT,REF_DP,REF_RV,REF_QUAL,ALT_DP,ALT_RV,ALT_QUAL,...,GFF_FEATURE,REF_CODON,REF_AA,ALT_CODON,ALT_AA,sample,path,outbreak,Sequence date,Sample collection date
0,NC_045512.2,854,C,T,3333,1468,49,2172,963,48,...,cds-YP_009724389.1,CCT,P,TCT,S,1110,/home/gk/analysis/2020.06.29.hCoV19/variants/i...,SD County,2020-06-27,2020-06-18
1,NC_045512.2,854,C,T,3333,1468,49,2172,963,48,...,cds-YP_009725295.1,CCT,P,TCT,S,1110,/home/gk/analysis/2020.06.29.hCoV19/variants/i...,SD County,2020-06-27,2020-06-18
2,NC_045512.2,27967,G,T,4074,2830,38,170,104,37,...,cds-YP_009724396.1,TGT,C,TTT,F,1110,/home/gk/analysis/2020.06.29.hCoV19/variants/i...,SD County,2020-06-27,2020-06-18
3,NC_045512.2,29715,G,T,19,13,45,1,0,72,...,NaN,NaN,NaN,NaN,NaN,1110,/home/gk/analysis/2020.06.29.hCoV19/variants/i...,SD County,2020-06-27,2020-06-18
4,NC_045512.2,5877,A,G,837,436,44,100,46,44,...,cds-YP_009724389.1,AAC,N,AGC,S,1927,/home/gk/analysis/2020.07.25.hCoV19_seq/varian...,SNF NEW,2020-07-24,2020-07-08


In [38]:
merged.to_csv("variants_outbreak_v2.csv", index=False)

In [39]:
print(merged.shape)
num_mutations = merged['POS'].unique().shape[0]
print(f'{num_mutations} Total Mutations')

(1684, 24)
982 Total Mutations


In [41]:
merged.loc[merged['outbreak']=='CALM', 'Sample collection date'].unique()

array(['2020-06-22', '2020-06-23', '2020-06-15', '2020-06-17',
       '2020-06-18', '2020-06-19'], dtype=object)

In [42]:
new_dates = ['2020-06-22', '2020-06-23', '2020-06-19']
merged.loc[(merged['outbreak']=='CALM') & (merged['Sample collection date'].isin(new_dates)), 'outbreak'] = 'CALM NEW'

In [44]:
merged.loc[merged['outbreak']=='SD County', 'Sample collection date'].unique()

array(['2020-06-18', '2020-06-21', '2020-06-25', '2020-06-23'],
      dtype=object)

In [45]:
new_dates = ['2020-06-25', '2020-06-23']
merged.loc[(merged['outbreak']=='SD County') & (merged['Sample collection date'].isin(new_dates)), 'outbreak'] = 'SD County NEW'

In [46]:
merged.to_csv("variants_outbreak_v2.csv", index=False)

In [47]:
import pandas as pd
import os
import sys
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fpath = "/Users/alaa/Documents/scripps/data/outbreak_aligned_clustered.clstr"
clusters = {}
current_cluster = None
with open(fpath) as f:
    for line in f.readlines():
        if line[0] == ">":
            clusters[line[1:-1]] = []
            current_cluster = line[1:-1]
        else:
            clusters[current_cluster].append(line.strip())
    f.close()

cluster_sizes = [len(v) for k,v in clusters.items()]
cluster_sizes = sorted(cluster_sizes)

rows = []
for k,v in clusters.items():
    for i in v:
        row = [k]
        row.extend(re.split(",|\t", i))
        rows.append(row)

df = pd.DataFrame.from_records(rows, columns=["cluster", "indice", "length", "name"])
df["length"] = df["length"].apply(lambda x: x[:-2]).astype(int)

df["seq_name"] = df["name"].apply(lambda x: re.search("(?<=\>)(.*?)[.]+", x).groups()[0])
df["seq_id"] = df["seq_name"].apply(lambda x: "-".join(x.split("-")[:2]))


In [57]:
outbreak_variants = pd.read_csv("variants_outbreak_v2.csv")
outbreak_variants["seq_id"] = outbreak_variants["path"].apply(lambda x: "-".join(os.path.basename(x).split("-")[:2]))

merged = pd.merge(outbreak_variants, df, on="seq_id", how ="inner")



In [58]:
merged.outbreak.value_counts()

SNF              1058
SNF NEW           394
SD County NEW      25
SD County          16
CALM NEW            7
CALM                3
Name: outbreak, dtype: int64

In [59]:

# # Get outbreak_variants names
# outbreak_ids = {
#     "CALM": [1076, 1066, 1093, 1085, 1090, 1099, 1104, 1044],
#     "SNF": [1940, 1941, 1954, 1959, 1960, 1964, 1970, 1957, 1951, 1955, 1961, 1952, 1963, 1948, 1950, 1953, 1962, 1971],
#     "SNF NEW": [1927, 1929, 1945, 1939, 1924, 1926, 1943, 1944, 1925, 1928, 1935, 1936, 1933, 1938, 1946, 1930, 1947, 1967],
#     "SD COUNTY": [1110, 1111, 1107, 1114, 1117, 1112, 1113, 1115, 1108, 1118]
# }

# outbreak_ids = dict([[k, ["SEARCH-{}".format(i) for i in v]] for k,v in outbreak_ids.items()])
# merged["outbreak"] = merged["seq_id"].apply(lambda x: [k for k,v in outbreak_ids.items() if x in v][0])

# Remove from different GFF features
merged = merged.drop_duplicates(["POS", "ALT", "seq_id"])
# Filter isnv frequencesi
merged = merged[(merged["ALT_FREQ"] <= 0.5) & (merged["ALT_FREQ"] >= 0.03)]

common_pos = []
for n, grp in merged.groupby(["POS"]):
    if grp["outbreak"].unique().shape[0] > 1:
        common_pos.append(n)

common_variants = merged[merged["POS"].isin(common_pos)]

cmap = plt.get_cmap("Set1")
outbreak_names = merged["outbreak"].unique().tolist()


In [60]:
outbreak_names

['SD County', 'SNF NEW', 'SNF', 'CALM NEW', 'CALM', 'SD County NEW']

In [61]:
nrow = 10
ncol = 10
f = plt.figure(figsize=(40, 40))
gs = gridspec.GridSpec(nrow, ncol)
for ind, (n, grp) in enumerate(common_variants.groupby("POS")):
#     print(n)
    ax = plt.subplot(gs[int(ind/nrow), int(ind % nrow)])
    d = [[i, igrp["ALT_FREQ"].tolist()] for i, igrp in grp.groupby("outbreak")]
    pos = [outbreak_names.index(i[0]) + 1 for i in d]
    ax.boxplot([i[1] for i in d], positions = pos, labels = [i[0] for i in d])
    ax.set_title("POS: {} SAMPLES: {}".format(n, grp["seq_id"].unique().shape[0]))
    ax.set_ylim([0.03, 0.5])

plt.tight_layout()
plt.savefig("../variants_freq_outbreak.pdf")
plt.close()


In [72]:
df[df['seq_id']=='SEARCH-1957'][['seq_name', 'cluster']]

,seq_name,cluster
13,SEARCH-1957-SAN,Cluster 1


In [71]:
df['seq_id']

0     SEARCH-1044
1     SEARCH-1066
2     SEARCH-1076
3     SEARCH-1085
4     SEARCH-1090
         ...     
69    SEARCH-2979
70    SEARCH-2982
71    SEARCH-1927
72    SEARCH-1948
73    SEARCH-1959
Name: seq_id, Length: 74, dtype: object

In [74]:
merged.outbreak.value_counts()

SNF              746
SNF NEW          303
SD County NEW     20
SD County         12
CALM NEW           7
CALM               2
Name: outbreak, dtype: int64

In [73]:
# Filter down to one cluster with identical sequences
identical_isnvs = merged[merged["cluster"] == "Cluster 1"]

isnv_counts = identical_isnvs.groupby(["POS"]).size()


f = plt.figure(figsize=(5, 10))
gs = gridspec.GridSpec(3, 1)
for ind, (n, grp) in enumerate(identical_isnvs[identical_isnvs["POS"].isin(isnv_counts[isnv_counts > 1].index.values)].groupby("POS")):
    ax = plt.subplot(gs[ind])
    d = [[i, igrp["ALT_FREQ"].tolist()] for i, igrp in grp.groupby("outbreak")]
    pos = [outbreak_names.index(i[0]) + 1 for i in d]
    ax.boxplot([i[1] for i in d], positions = pos, labels = [i[0] for i in d])
    ax.set_title("POS: {} SAMPLES: {}".format(n, grp["seq_id"].unique().shape[0]))
    ax.set_ylim([0, 0.5])

plt.tight_layout()
plt.savefig("../variants_freq_outbreak_identical.pdf")
plt.close()


### POS Frequencies per Outbreak

In [22]:
# number of samples per outbreak
variants_df.groupby('outbreak').agg(num_samples=('sample', 'nunique'))

,num_samples
outbreak,
CALM,8
SD County,10
SNF,18
SNF NEW,18


In [95]:
# 15 most frequent variants per outbreak
topvars_per_outbreak = (variants_df.loc[variants_df['ALT_FREQ'] < 1]
                        .groupby(['outbreak', 'POS'])
                        .agg(num=('POS', 'size')))
topvars_per_outbreak.reset_index().groupby('outbreak').apply(lambda x: x.nlargest(15, 'num'))

outbreak    POS  num
outbreak                             
CALM      7          CALM   2143   14
          17         CALM  11575   14
          14         CALM   7768   12
          2          CALM    241    8
          22         CALM  14408    8
          26         CALM  20268    7
          28         CALM  23403    7
          41         CALM  28854    6
          9          CALM   3037    4
          3          CALM    769    2
          4          CALM   1614    2
          6          CALM   2089    2
          8          CALM   2220    2
          10         CALM   3144    2
          11         CALM   4992    2
SD County 64    SD County  11575   20
          51    SD County   2143   16
          62    SD County   7768   14
          68    SD County  14408   10
          78    SD County  23403   10
          73    SD County  20268    9
          81    SD County  28854    9
          61    SD County   6636    8
          47    SD County    241    3
          48    SD County    854    2
          49    SD County   1101    2
          50    SD County   1233    2
          52    SD County   2834    2
          53    SD County   3443    2
          54    SD County   3583    2
SNF       406         SNF  11575   28
          117         SNF   2143   20
          518         SNF  20268   17
          638         SNF  23403   16
          436         SNF  14408   11
          773         SNF  28854   11
          470         SNF  16708   10
          93          SNF    241    7
          178         SNF   3157    6
          187         SNF   3549    6
          317         SNF   7252    6
          118         SNF   2246    4
          131         SNF   2395    4
          136         SNF   2460    4
          149         SNF   2549    4
SNF NEW   901     SNF NEW  11575   32
          800     SNF NEW   2143   26
          1029    SNF NEW  23403   14
          1117    SNF NEW  28854   14
          924     SNF NEW  16708   13
          974     SNF NEW  20268   12
          865     SNF NEW   6696    5
          790     SNF NEW    241    4
          792     SNF NEW    664    4
          815     SNF NEW   3119    4
          830     SNF NEW   5048    4
          857     SNF NEW   5744    4
          858     SNF NEW   5877    4
          861     SNF NEW   6283    4
          873     SNF NEW   7053    4

In [83]:
# tmp2 = (variants_df.loc[variants_df['ALT_FREQ'] < 0.5]
#         .groupby(['outbreak', 'POS'])
#         .size()
#         .to_frame()
#         .rename(columns={0: 'num'}))
# tmp2

In [84]:
# tmp3 = tmp1 == tmp2
# tmp3.loc[tmp3['num']==False]

From the above, let us eyeball and pick the following as **positions of interest** based on their prevalence across all outbreaks:
* 2143
* 3037
* 7768
* 11575
* 241

In [85]:
def generate_pos_boxplots(variants_df: pd.DataFrame, poi: list):
    topvars_df = variants_df.loc[variants_df['POS'].isin(poi)]
    outbreaks = topvars_df['outbreak'].unique()
    layout = go.Layout(xaxis=dict(type='category'))
    fig = go.Figure(layout=layout)
    for ob in outbreaks:
        y_i = topvars_df.loc[topvars_df['outbreak'] == ob]
        Q1 = y_i['ALT_FREQ'].quantile(0.25)
        Q3 = y_i['ALT_FREQ'].quantile(0.75)
        IQR = Q3 - Q1    #IQR is interquartile range. 
        outlier_filter = (y_i['ALT_FREQ'] >= Q1 - 1.5 * IQR) & (y_i['ALT_FREQ'] <= Q3 + 1.5 *IQR)
        y_i = y_i.loc[outlier_filter]
        fig.add_trace(go.Box(y=y_i['ALT_FREQ'], x=y_i['POS'], name=ob))
    return fig

In [87]:
poi = [11575, 20268]

fig = generate_pos_boxplots(variants_df, poi)
fig.show()

In [88]:
poi = [2143, 28854, 241, 23403]
fig = generate_pos_boxplots(variants_df, poi)
fig.show()

In [89]:
variants_df['outbreak'].unique()

array(['SD County', 'SNF NEW', 'SNF', 'CALM'], dtype=object)

**Applying Levene's test for heteroscedasticity between each outbreak population**

In [90]:
positions = variants_df['POS'].unique().astype(int)
pvals = np.zeros_like(positions, dtype=float)
test_stats = np.zeros_like(positions, dtype=float)
for i, pos in enumerate(positions):
    calm_pop = variants_df.loc[(variants_df['POS']==pos) & (variants_df['outbreak']=='CALM')]
    snf_pop = variants_df.loc[(variants_df['POS']==pos) & (variants_df['outbreak']=='SNF')]
    snf2_pop = variants_df.loc[(variants_df['POS']==pos) & (variants_df['outbreak']=='SNF NEW')]
    sd_pop = variants_df.loc[(variants_df['POS']==pos) & (variants_df['outbreak']=='SD County')]
    w, p = levene(calm_pop['ALT_FREQ'], snf_pop['ALT_FREQ'], snf2_pop['ALT_FREQ'], 
                     sd_pop['ALT_FREQ'], center='median')
#     print(i, w)
    test_stats[i] = w
    pvals[i] = p
    
results = pd.DataFrame(data=np.vstack((positions, test_stats, pvals)).T, 
                       columns=['POS', 'statistic', 'pval'])
results['POS'] = results['POS'].astype(int)
results.head()

/Users/alaa/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning:

Mean of empty slice.

/Users/alaa/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars



,POS,statistic,pval
0,241,0.700257,0.556341
1,854,NaN,NaN
2,2143,2.119650,0.102207
3,3037,1.286598,0.282959
4,7768,1.268683,0.289057


In [91]:
results.shape

(1027, 3)

In [92]:
results.dropna()

,POS,statistic,pval
0,241,0.700257,0.556341
2,2143,2.119650,0.102207
3,3037,1.286598,0.282959
4,7768,1.268683,0.289057
5,11575,3.561216,0.016866
6,14408,1.501530,0.225812
7,20268,6.464767,0.000895
8,23403,1.270800,0.294499
10,28854,0.951865,0.422935


## Looking at top 5 variants for each sample

In [522]:
top_vars = variants_df.groupby(['sample']).apply(lambda x: x.nlargest(5,['ALT_FREQ'])).reset_index(drop=True)[['sample', 'POS', 'REF', 'ALT', 'outbreak']]
top_vars = top_vars.groupby('sample').agg(top_vars=('POS', 'unique')).reset_index()

In [523]:
top_vars['key'] = 0
top_vars_matrix = pd.merge(top_vars, top_vars, on='key', how='outer')

In [524]:
top_vars_matrix

,sample_x,top_vars_x,key,sample_y,top_vars_y
0,1044,"[3037, 7768, 23403]",0,1044,"[3037, 7768, 23403]"
1,1044,"[3037, 7768, 23403]",0,1066,"[3037, 28854, 7768]"
2,1044,"[3037, 7768, 23403]",0,1076,"[3037, 14408, 2143]"
3,1044,"[3037, 7768, 23403]",0,1085,"[3037, 28854, 23403, 20268]"
4,1044,"[3037, 7768, 23403]",0,1090,"[14408, 28854, 3037, 241]"
...,...,...,...,...,...
2911,1971,"[2143, 3037, 7768]",0,1963,"[3037, 7768, 28854]"
2912,1971,"[2143, 3037, 7768]",0,1964,"[241, 2143, 3037]"
2913,1971,"[2143, 3037, 7768]",0,1967,"[241, 3037, 7768]"
2914,1971,"[2143, 3037, 7768]",0,1970,"[3037, 7768, 28854]"


In [525]:
top_vars_matrix['common_vars'] = top_vars_matrix.apply(lambda x: compute_common_variants(x, 'top_vars_x', 'top_vars_y'), axis=1)
top_vars_matrix['num_common_vars'] = top_vars_matrix['common_vars'].str.len()
top_vars_matrix['label_x'] = top_vars_matrix['sample_x'].apply(label_sample)
top_vars_matrix['label_y'] = top_vars_matrix['sample_y'].apply(label_sample)

In [526]:
variants_df.loc[variants_df['sample']==1085].nlargest(5, 'ALT_FREQ')

,REGION,POS,REF,ALT,REF_DP,REF_RV,REF_QUAL,ALT_DP,ALT_RV,ALT_QUAL,...,PVAL,PASS,GFF_FEATURE,REF_CODON,REF_AA,ALT_CODON,ALT_AA,sample,path,outbreak
3,NC_045512.2,3037,C,T,0,0,0,448,164,47,...,2.422280e-314,True,cds-YP_009724389.1,TTC,F,TTT,F,1085,/home/gk/analysis/2020.06.29.hCoV19/variants/i...,CALM
4,NC_045512.2,3037,C,T,0,0,0,448,164,47,...,2.422280e-314,True,cds-YP_009725295.1,TTC,F,TTT,F,1085,/home/gk/analysis/2020.06.29.hCoV19/variants/i...,CALM
14,NC_045512.2,28854,C,T,1,0,36,14985,4308,37,...,0.000000e+00,True,cds-YP_009724397.2,TCA,S,TTA,L,1085,/home/gk/analysis/2020.06.29.hCoV19/variants/i...,CALM
12,NC_045512.2,23403,A,G,1,0,36,4662,2586,45,...,0.000000e+00,True,cds-YP_009724390.1,GAT,D,GGT,G,1085,/home/gk/analysis/2020.06.29.hCoV19/variants/i...,CALM
11,NC_045512.2,20268,A,G,1,0,36,3964,1928,38,...,0.000000e+00,True,cds-YP_009724389.1,TTA,L,TTG,L,1085,/home/gk/analysis/2020.06.29.hCoV19/variants/i...,CALM


In [490]:
top_vars_matrix.loc[top_vars_matrix['sample_x']==1085].sort_values(['sample_x', 'num_common_vars'], ascending=True)

,sample_x,top_vars_x,key,sample_y,top_vars_y,common_vars,num_common_vars,label_x,label_y
173,1085,"[3037, 28854, 23403, 20268]",0,1111,"[241, 2143, 2596]",{},0,CALM,SD County
183,1085,"[3037, 28854, 23403, 20268]",0,1927,"[241, 2143, 7768]",{},0,CALM,SNF NEW
164,1085,"[3037, 28854, 23403, 20268]",0,1076,"[3037, 14408, 2143]",{3037},1,CALM,CALM
168,1085,"[3037, 28854, 23403, 20268]",0,1099,"[2143, 3037, 7768]",{3037},1,CALM,CALM
170,1085,"[3037, 28854, 23403, 20268]",0,1107,"[241, 2596, 3037]",{3037},1,CALM,SD County
176,1085,"[3037, 28854, 23403, 20268]",0,1114,"[2596, 3037, 14408]",{3037},1,CALM,SD County
177,1085,"[3037, 28854, 23403, 20268]",0,1115,"[3037, 6636, 7768]",{3037},1,CALM,SD County
178,1085,"[3037, 28854, 23403, 20268]",0,1117,"[241, 2143, 3037]",{3037},1,CALM,SD County
180,1085,"[3037, 28854, 23403, 20268]",0,1924,"[241, 3037, 7768]",{3037},1,CALM,SNF NEW
181,1085,"[3037, 28854, 23403, 20268]",0,1925,"[241, 2143, 3037]",{3037},1,CALM,SNF NEW


## Looking at common intra-host mutations across samples

In [413]:
# freq_filter = variants_df['ALT_FREQ'] < 0.75
mutations_per_sample = variants_df.groupby('sample').agg(MUTS=('POS', 'unique'), ENT=('ALT_FREQ', entropy)).reset_index()

In [414]:
mutations_per_sample.shape

(54, 3)

In [416]:
mutations_per_sample['key'] = 0
mutations_matrix = pd.merge(mutations_per_sample, mutations_per_sample, on='key', how='outer')

In [481]:
def compute_common_variants(x, col_x, col_y):
    var_x = set(x[col_x])
    var_y = set(x[col_y])
    return var_x.intersection(var_y)

mutations_matrix['common_vars'] = mutations_matrix.apply(lambda x: compute_common_variants(x, 'MUTS_x', 'MUTS_y'), axis=1)
mutations_matrix['num_common_vars'] = mutations_matrix['common_vars'].str.len()
mutations_matrix['ENT_delta'] = abs(mutations_matrix['ENT_x'] - mutations_matrix['ENT_y'])

In [419]:
def label_sample(x):
    if x in calm_ids:
        return 'CALM'
    elif x in snf_ids:
        return 'SNF'
    elif x in snf_new_ids:
        return 'SNF NEW'
    else: 
        return 'SD County'

In [420]:
mutations_matrix['label_x'] = mutations_matrix['sample_x'].apply(label_sample)
mutations_matrix['label_y'] = mutations_matrix['sample_y'].apply(label_sample)

In [111]:
# TODO: look at spread of frequency per position
mutations_matrix.loc[mutations_matrix['sample_x']==1076].sort_values(['sample_x', 'num_common_vars'], ascending=False)

NameError: name 'mutations_matrix' is not defined

## Outbreak Identification using Unsupervised Clustering

### Main Steps
* load the variants for each sample into dataframe
* associate each sample with corresponding outbreak
* Embed variants for each sample into vectorized form
* Apply unsupervised clustering techniques to identify outbreaks

### Features
* Mean and SD frequency of minor variants per sample
* Number of distinct variants per sample
* Number of `indel` variants per sample

In [425]:
# variants_df.loc[(variants_df['sample']==1066)]

In [426]:
variants_df['IS_SYN'] = 1
variants_df.loc[variants_df['REF_AA'] != variants_df['ALT_AA'], 'IS_SYN'] = 0

In [427]:
variants_df['VAR'] = variants_df['POS'].map(str) + '-' + variants_df['ALT']
freq_filter = (variants_df['ALT_FREQ'] < 0.5)
freq_stats = (variants_df.loc[freq_filter]
              .groupby('sample')
              .agg(AVG_FREQ_MINOR=('ALT_FREQ', 'mean'), 
                   NUM_VARS_MINOR=('VAR', 'nunique'),
                   NUM_SYN_MINOR=('IS_SYN', 'sum'))
              .fillna(0))
var_stats = (variants_df.groupby('sample')
             .agg(
                  NUM_VARS=('VAR', 'nunique'), 
                  NUM_SYN=('IS_SYN', 'sum'),
                  #NUM_INDELS=('is_indel', 'sum'), 
                  AVG_FREQ=('ALT_FREQ', 'mean'),
                  ENT_FREQ=('ALT_FREQ', entropy),
             ))

In [428]:
stats_per_sample = pd.merge(var_stats, freq_stats, on='sample', how='left').fillna(0)

In [293]:
stats_per_sample.head()

,NUM_VARS,NUM_SYN,AVG_FREQ,ENT_FREQ,AVG_FREQ_MINOR,NUM_VARS_MINOR,NUM_SYN_MINOR
sample,,,,,,,
1044,16,9,0.745533,2.866813,0.050864,4.0,0.0
1066,9,9,0.997380,2.564934,0.000000,0.0,0.0
1076,10,9,0.932797,2.585111,0.072664,1.0,0.0
1085,11,10,0.877818,2.616021,0.102778,2.0,1.0
1090,10,9,0.927148,2.578732,0.043478,1.0,0.0


In [429]:
def get_cluster_labels(clusters: list, stats_per_sample: pd.DataFrame):
    out_clusters = collections.defaultdict(list)
    for i, c in enumerate(clusters):
        for idx in c[0]:
            sample_id = stats_per_sample.index[idx]
            out_clusters[i].append(sample_id)
    return out_clusters

In [430]:
# Number of grade levels (e.g. A, B, C, F)
k = 4
from sklearn import preprocessing
X = stats_per_sample.values
X = preprocessing.scale(X, axis=0)
# Use K-Means
centroids, clusters = kmeans(X, k=k)
clusters = get_cluster_labels(clusters, stats_per_sample)
print(f"Centroids: {np.mean(centroids, axis=1)}")
print(f"Clusters: {clusters}")

Centroids: [ 0.91139476 -0.51540478  0.02316929 -0.24553755]
Clusters: defaultdict(<class 'list'>, {0: [1930, 1936, 1940, 1941, 1945, 1946, 1950, 1953, 1954, 1960, 1964], 1: [1066, 1093, 1112, 1115], 2: [1085, 1110, 1111, 1924, 1929, 1951], 3: [1044, 1076, 1090, 1099, 1104, 1107, 1108, 1113, 1114, 1117, 1118, 1925, 1926, 1927, 1928, 1933, 1935, 1938, 1939, 1943, 1944, 1947, 1948, 1952, 1955, 1957, 1959, 1961, 1962, 1963, 1967, 1970, 1971]})


In [431]:
variants_df['sample'].unique()

array([1110, 1927, 1940, 1941, 1954, 1959, 1960, 1964, 1970, 1957, 1929,
       1951, 1076, 1066, 1945, 1955, 1093, 1939, 1961, 1085, 1952, 1090,
       1963, 1099, 1104, 1044, 1924, 1926, 1943, 1944, 1948, 1950, 1953,
       1962, 1925, 1928, 1935, 1936, 1111, 1107, 1114, 1933, 1938, 1946,
       1971, 1930, 1947, 1967, 1117, 1112, 1113, 1115, 1108, 1118])

In [96]:
variants_df['is_indel'] = False
variants_df.loc[(variants_df['ALT'].str.contains('\+')), 'is_indel'] = True

In [97]:
variants_df.groupby('outbreak').agg(indel_count=('is_indel', 'sum'), variant_count=('POS', 'nunique'), num_samples=('sample', 'nunique'))

,indel_count,variant_count,num_samples
outbreak,,,
CALM,2.0,46,8
SD County,1.0,43,10
SNF,23.0,706,18
SNF NEW,12.0,342,18


In [98]:
variants_df['is_indel'].value_counts()

False    2428
True       38
Name: is_indel, dtype: int64

In [99]:
from scipy.stats import entropy
def get_unique_vals(x):
    x = x.unique()
    return list(x)

def get_unique_counts(x):
    _, counts = np.unique(x, return_counts=True)
    return list(counts)

In [100]:
### Point Mutations per location
mdf = (variants_df.groupby(['outbreak', 'POS'])
            .agg(ref=('REF', get_unique_vals),
                 ref_aa=('REF_AA', get_unique_vals),
                 count=('POS', 'count'),
                 alts=('ALT', get_unique_vals),
                 alts_aa=('ALT_AA', get_unique_vals),
                 alt_cnts=('ALT', get_unique_counts),
                 alts_num=('ALT', 'nunique'),
                 entropy=('ALT_FREQ', entropy),
                 alt_freq=('ALT_FREQ', 'mean'))
            .sort_values('alts_num', ascending=False)
            .reset_index())

In [101]:
mdf.outbreak.unique()

array(['SNF', 'SNF NEW', 'SD County', 'CALM'], dtype=object)

In [102]:
samples_per_loc = variants_df.groupby('outbreak').agg(sample_num=('sample', 'nunique')).reset_index()
samples_per_loc

,outbreak,sample_num
0,CALM,8
1,SD County,10
2,SNF,18
3,SNF NEW,18


In [103]:
mdf = pd.merge(mdf, samples_per_loc, on='outbreak')
mdf['count_normed'] = mdf['count'] / mdf['sample_num']

In [104]:
print(mdf.shape)
mdf.sort_values(['count_normed', 'alts_num'], ascending=False).head()

(1137, 13)


,outbreak,POS,ref,ref_aa,count,alts,alts_aa,alt_cnts,alts_num,entropy,alt_freq,sample_num,count_normed
295,SNF,3037,[C],[F],36,[T],[F],[36],1,3.583519,1.000000,18,2.0
375,SNF,2143,[C],[P],36,[T],[P],[36],1,3.583519,0.999560,18,2.0
596,SNF,7768,[C],[I],36,[T],[I],[36],1,3.583519,0.999745,18,2.0
657,SNF,11575,[C],[F],36,[T],[F],[36],1,3.583414,0.990052,18,2.0
719,SNF NEW,3037,[C],[F],36,[T],[F],[36],1,3.583516,0.999450,18,2.0


In [110]:
# mdf

In [108]:
# Mutation Prevalence versus Diversity
def generate_freq_plot(mdf: pd.DataFrame):
    fig = go.Figure()
    locations = mdf['outbreak'].unique()
    for loc in locations:
        fig.add_trace(go.Scatter(
            x=mdf.loc[mdf['outbreak']==loc, 'POS'],
            y=mdf.loc[mdf['outbreak']==loc, 'alt_freq'],
            mode='markers',
            text=mdf.loc[mdf['outbreak']==loc, ['POS', 'ref', 'alts', 'ref_aa', 'alts_aa']],
            hovertemplate =
            "POS: <b>%{text[0]}</b><br>" +
            "REF: <b>%{text[1]}</b><br>" + 
            "ALTS: <b>%{text[2]}</b><br>" +
            "REF_AA: <b>%{text[3]}</b><br>" +
            "ALTS_AA: <b>%{text[4]}</b><br>",
            name=f'{loc}'
        ))

    fig.update_layout(title=f'Mutation Freq and Position',
              xaxis_title="Position of Mutation",
              yaxis_title="Mutation Frequency",
              template='plotly',
              height=800)
    return fig

fig = generate_freq_plot(mdf)
fig.show()

In [109]:
# Mutation Prevalence versus Diversity
def generate_entropy_plot(mdf: pd.DataFrame):
    fig = go.Figure()
    locations = mdf['outbreak'].unique()
    for loc in locations:
        fig.add_trace(go.Scatter(
            x=mdf.loc[mdf['outbreak']==loc, 'count_normed'],
            y=mdf.loc[mdf['outbreak']==loc, 'entropy'],
            mode='markers',
            text=mdf.loc[mdf['outbreak']==loc, ['POS', 'ref', 'alts', 'ref_aa', 'alts_aa']],
            hovertemplate =
            "POS: <b>%{text[0]}</b><br>" +
            "REF: <b>%{text[1]}</b><br>" + 
            "ALTS: <b>%{text[2]}</b><br>" +
            "REF_AA: <b>%{text[3]}</b><br>" +
            "ALTS_AA: <b>%{text[4]}</b><br>",
            name=f'{loc}'
        ))

    fig.update_layout(title=f'Mutation Entropy and Prevalence',
              xaxis_title="Number of Samples with Mutation",
              yaxis_title="Mutation Entropy",
              template='plotly',
              height=800)
    return fig

fig = generate_entropy_plot(mdf)
fig.show()

# Variant Analysis
This notebook contains developmental code for the analysis of intra-host genomic variants from Sars-Cov-2 samples. 

## Progress:
* Merged variants data from `analysis` and `metadata`
    * still need to account for missing samples due to naming inconsistencies
* Analysis of Point Mutations
* Analysis of iSNVs per location
* Analysis of top iSNVs per location over time


* Look at sequences from SNF and CALM Outbreaks
    * they have identical consensus sequences
    * can we use intra-host variants to differentiate between them?
* Repeat analysis with deletions only
* filter variants that have < 0.5 `ALT_FREQ`
* For each variant, display box plot of frequency over time 
    * generate equivalent plot for variants with insertions/deletions
    * number of deletions per ORF
    * generate boxplot for the 3 variants found on New Orleans (green, purple, orange)
        * we want to look at their relationship
* Install Genious Prime using server details from Slack (KG chat)
    * look at New Orleans genome to figure out variant co-occurences

* Incorporate Shannon Entropy into Mutations visualization
* Generate plot for iSNVs with highest rate of prevalence increase with time
* Repeat analysis once missing samples are accounted for
* Create GUI that allows user to filter by location and mutation type
* Create automated pipeline for updating metadata 
* Get feedback on visuals 


## Questions
* we always assume single-nucleotide variants - what about co-occurences???
    * its called phase sequencing (identifying multiple variants on same reads)
* how can we automate metadata updates?
* metadata contains missing collection dates?
